In [60]:
import os
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import date
import sys
from datetime import datetime , timedelta, date
import time
from datetime import date
from dateutil.relativedelta import relativedelta
from pathlib import Path
from glob import glob 
import shutil

In [73]:
PATH_PASTA_ARQUIVOS = r"C:\Users\thiag\Documents\BASE_SUSEP\Download_arquivo"
PATH_PASTA_ARQUIVOS_CSV = r"C:\Users\thiag\Documents\BASE_SUSEP\Arquivos_CSV"
QDE_MAX_SEGUNDOS_DOWNLOAD = 600 



In [61]:
def criar_pasta(path):
    try:
        shutil.rmtree(path,ignore_errors=True)
    except:
        ...

    os.makedirs(path,exist_ok=True)

In [ ]:
def buscar_arquivo_na_susep(path:Path,qde_max_segundos_download:int):
    check_download = False
    qde_max_segundos = qde_max_segundos_download
    criar_pasta(PATH_PASTA_ARQUIVOS)
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
        "download.default_directory": path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    options.add_argument("--headless")

    servico = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=servico,options=options)
    driver.get(r'https://www2.susep.gov.br/menuestatistica/ses/principal.aspx')

    time.sleep(5)

    valor = driver.find_element(By.ID,'GEEST')
    if valor:
        driver.get(r"https://www2.susep.gov.br/redarq.asp?arq=BaseCompleta%2ezip")
    else: 
        print('link não encontrado')
        driver.close()
        return
    time.sleep(5)
    segundos = 0
    
    while True:
        if segundos % 10 == 0:
                print('Efetuando Download')
        
        segundos += 1
        if segundos > qde_max_segundos: 
            print('Excedeu o tempo limite de download')
            driver.close()
            return check_download
        
        arquivo = glob(os.path.join(path,'*.crdownload'))
        if arquivo:
            time.sleep(1)
            continue
        break
    
    print('Download Concluído')
    check_download = 1
    driver.close()
    check_download = True
    return check_download

In [78]:

check_download = buscar_arquivo_na_susep(path=PATH_PASTA_ARQUIVOS,
                        qde_max_segundos_download=QDE_MAX_SEGUNDOS_DOWNLOAD)

Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download
Efetuando Download
Download Concluído


In [87]:
import zipfile

In [94]:
def descompactar_zip(path_origem,check_sucesso:bool = False):
    arquivos_csv =  glob(os.path.join(path_origem,'*.csv'))
    for arq in arquivos_csv:
        os.remove(arq)
    time.sleep(5)
    if check_sucesso:
        arquivos = glob(os.path.join(path_origem,'*.zip'))

        for arquivo in arquivos:
            if arquivo.endswith('.zip'):
                caminho_completo = os.path.join(path_origem, arquivo)

                with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
                    zip_ref.extractall(path_origem)

    else:
        print('Erro')

In [95]:
descompactar_zip(path_origem=PATH_PASTA_ARQUIVOS,check_sucesso=check_download)